In [5]:
import xml.etree.ElementTree as ET
import os
from pathlib import Path

def process_xsd_file(xsd_file, output_file=None):
    """
    Processes XSD file to change dateTime types to date types for columns 
    with DATE columnDefinition in Appian JPA annotations.
    
    Args:
        xsd_file (str): Path to input XSD file
        output_file (str, optional): Path to output file. If None, creates 'modified_' prefix
    
    Returns:
        bool: True if successful, False otherwise
    """
    if not os.path.exists(xsd_file):
        print(f"Error: File {xsd_file} not found")
        return False
    
    try:
        tree = ET.parse(xsd_file)
        root = tree.getroot()
        changes_made = 0
        
        for element in root.iter('{http://www.w3.org/2001/XMLSchema}element'):
            appinfo = element.find('{http://www.w3.org/2001/XMLSchema}annotation/'
                                   '{http://www.w3.org/2001/XMLSchema}appinfo')
            
            if appinfo is not None and appinfo.text and 'columnDefinition="DATE"' in appinfo.text:
                current_type = element.get('type')
                if current_type == 'xsd:dateTime':
                    element.set('type', 'xsd:date')
                    changes_made += 1
                    print(f"Changed element '{element.get('name')}' from dateTime to date")
        
        if output_file is None:
            input_path = Path(xsd_file)
            output_file = input_path.parent / f"modified_{input_path.name}"
        
        tree.write(output_file, encoding='utf-8', xml_declaration=True)
        print(f"✅ Successfully processed {changes_made} elements")
        print(f"✅ Output written to: {output_file}")
        return True
        
    except Exception as e:
        print(f"❌ Error processing XSD file: {e}")
        return False

def main():
    # Example usage
    xsd_file = r"path/to/your/xsd_file.xsd"
    
    # Process with default output name
    success = process_xsd_file(xsd_file)
    
    # Or specify custom output
    # success = process_xsd_file(xsd_file, "custom_output.xsd")
    
    if success:
        print("🎉 XSD processing completed successfully!")
    else:
        print("❌ XSD processing failed!")

if __name__ == "__main__":
    main()

Error: File path/to/your/xsd_file.xsd not found
❌ XSD processing failed!


In [6]:
# sample XSD for testing

def create_sample_xsd():
    """
    Creates a sample XSD file for testing the converter.
    """
    sample_xsd = '''<?xml version="1.0" encoding="UTF-8"?>
<xsd:schema xmlns:xsd="http://www.w3.org/2001/XMLSchema" 
            targetNamespace="http://www.appian.com/ae/types/2009" 
            xmlns:tns="http://www.appian.com/ae/types/2009" 
            elementFormDefault="qualified">

    <xsd:element name="Transaction" type="tns:Transaction"/>
    
    <xsd:complexType name="Transaction">
        <xsd:all>
            <!-- This should be changed: dateTime with DATE columnDefinition -->
            <xsd:element name="transactionDate" nillable="true" type="xsd:dateTime">
                <xsd:annotation>
                    <xsd:appinfo source="appian.jpa">@Column(name="TRANSACTION_DATE", columnDefinition="DATE")</xsd:appinfo>
                </xsd:annotation>
            </xsd:element>
            
            <!-- This should be changed: dateTime with DATE columnDefinition -->
            <xsd:element name="valueDateAcctDebitDate" nillable="true" type="xsd:dateTime">
                <xsd:annotation>
                    <xsd:appinfo source="appian.jpa">@Column(name="VALUEDATEACCTDEBITDATE", columnDefinition="DATE")</xsd:appinfo>
                </xsd:annotation>
            </xsd:element>
            
            <!-- This should NOT be changed: dateTime without DATE columnDefinition -->
            <xsd:element name="createdTimestamp" nillable="true" type="xsd:dateTime">
                <xsd:annotation>
                    <xsd:appinfo source="appian.jpa">@Column(name="CREATED_TIMESTAMP")</xsd:appinfo>
                </xsd:annotation>
            </xsd:element>
            
        </xsd:all>
    </xsd:complexType>
    
</xsd:schema>'''

    # Save sample XSD file
    with open('sample_transaction.xsd', 'w', encoding='utf-8') as f:
        f.write(sample_xsd)

    print("✅ Sample XSD file created: sample_transaction.xsd")
    print("📋 This file contains:")
    print("  - 2 elements with dateTime type and DATE columnDefinition (should be changed)")
    print("  - 1 element with dateTime type without DATE columnDefinition (should stay unchanged)")

# Create sample file for testing
create_sample_xsd()

✅ Sample XSD file created: sample_transaction.xsd
📋 This file contains:
  - 2 elements with dateTime type and DATE columnDefinition (should be changed)
  - 1 element with dateTime type without DATE columnDefinition (should stay unchanged)


In [4]:
# Test the converter with sample file
def test_converter():
    """
    Tests the XSD converter with the sample file.
    """
    print("🧪 Testing XSD converter with sample file...")
    
    # Test the converter
    success = process_xsd_file('sample_transaction.xsd', 'fixed_transaction.xsd')
    
    if success:
        print("\n🔍 Checking the results:")
        
        # Read and check the modified file
        with open('fixed_transaction.xsd', 'r', encoding='utf-8') as f:
            content = f.read()
            
        # Check which elements were changed
        print("📊 Results:")
        print("  - transactionDate type:", 
              "xsd:date ✅" if 'transactionDate" nillable="true" type="xsd:date"' in content else "xsd:dateTime ❌")
        print("  - valueDateAcctDebitDate type:", 
              "xsd:date ✅" if 'valueDateAcctDebitDate" nillable="true" type="xsd:date"' in content else "xsd:dateTime ❌")
        print("  - createdTimestamp type:", 
              "xsd:dateTime ✅" if 'createdTimestamp" nillable="true" type="xsd:dateTime"' in content else "xsd:date ❌")
        
        print("\n✅ Test completed! Check 'fixed_transaction.xsd' for results.")
    else:
        print("❌ Test failed!")

# Run the test
test_converter()

🧪 Testing XSD converter with sample file...
Changed element 'transactionDate' from dateTime to date
Changed element 'valueDateAcctDebitDate' from dateTime to date
✅ Successfully processed 2 elements
✅ Output written to: fixed_transaction.xsd

🔍 Checking the results:
📊 Results:
  - transactionDate type: xsd:date ✅
  - valueDateAcctDebitDate type: xsd:date ✅
  - createdTimestamp type: xsd:dateTime ✅

✅ Test completed! Check 'fixed_transaction.xsd' for results.
